In [10]:
from langchain_ollama.llms import OllamaLLM
from langchain_ollama.chat_models import ChatOllama
model = "qwen2.5:0.5b"
llm = OllamaLLM(model=model)
llm.invoke("Tell me a joke")

"Sure! Here's one for you:\n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing!\n\nThis is a classic pun that uses a common phrase to make a point about taste perception and visual cues. I hope that's a funny joke for you!"

In [11]:
chat_model = ChatOllama(model=model)
response = llm.invoke("What is the capital of France?")
print(response)

The capital of France is Paris.


In [12]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content="Hello, how are you?")]
chat_response = chat_model.invoke(messages)
print(chat_response.content)

Hello! As an AI language model, I am here to help and provide information. How can I assist you today?


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")


In [ ]:
from langchain.messages import SystemMessage, HumanMessage, AIMessage

sys_msg = SystemMessage("You are a helpful assitant")
human_msg = HumanMessage("Hello, how are you?")

messages = [sys_msg, human_msg]
response = model.invoke